In [2]:
import pandas

data = pandas.read_csv('data-logistic.csv', header=None)
data.head()

,0,1,2
0,-1,-0.663827,-0.138526
1,1,1.994596,2.468025
2,-1,-1.247395,0.749425
3,1,2.309374,1.899836
4,1,0.849143,2.407750


In [5]:
y = data[0].copy()
X = data.copy()
del X[0]
X.head()

,1,2
0,-0.663827,-0.138526
1,1.994596,2.468025
2,-1.247395,0.749425
3,2.309374,1.899836
4,0.849143,2.407750


In [18]:
import numpy as np
from numpy.linalg import norm
from math import exp

def getSum(data, w1, w2, flag):
    l = data.shape[0]
    summ = 0
    
    for index, row in data.iterrows():
        yi = row[0]
        xi1 = row[1]
        xi2 = row[2]
        summ+= yi*row[flag]*(1 - 1/(1+exp(-yi*(w1*xi1+w2*xi2))  )  )
    
    return summ/l

def gradSpusk(data, epsilon=1e-5, k=0.1):
    w1 = 0
    w2 = 0
    norma = 1
    
    while (norma > epsilon):
        temp1 = w1
        temp2 = w2
        
        w1 += k*getSum(data, w1=temp1, w2=temp2, flag=1)
        w2 += k*getSum(data, w2=temp2, w1=temp1, flag=2)
        
        norma = norm(np.array([w1-temp1,w2-temp2]))
        
    return {'w1': w1, 'w2': w2}
        
print( gradSpusk(data) )

{'w1': 0.2878116204717764, 'w2': 0.09198330215925439}


In [19]:
def gradSpuskL2(data, epsilon=1e-5, k=0.1, C=10):
    w1 = 0
    w2 = 0
    norma = 1
    
    while (norma > epsilon):
        temp1 = w1
        temp2 = w2
        
        w1 += k*getSum(data, w1=temp1, w2=temp2, flag=1) - k*C*temp1
        w2 += k*getSum(data, w2=temp2, w1=temp1, flag=2) - k*C*temp2
        
        norma = norm(np.array([w1-temp1,w2-temp2]))
        
    return {'w1l2': w1, 'w2l2': w2}

print( gradSpuskL2(data) )

{'w1l2': 0.028558754546234223, 'w2l2': 0.02478013724973556}


In [23]:
from sklearn.metrics import roc_auc_score

w1 = 0.2878116204717764
w2 = 0.09198330215925439
w1l2 = 0.028558754546234223
w2l2 = 0.02478013724973556

def getProbs(w1, w2, data):
    probs = []
    for index, row in data.iterrows():
        yi = row[0]
        xi1 = row[1]
        xi2 = row[2]
        prob = 1/(1 + exp(-(xi1*w1 + xi2*w2)))
        probs.append(prob)
        
    return probs

probs = getProbs(w1, w2, data)
roc_auc_score(y, probs) # 0.927

0.9268571428571428

In [24]:
probsL2 = getProbs(w1l2, w2l2, data)
roc_auc_score(y, probsL2) # 0.936

0.9362857142857142

In [25]:
with open('asd.txt', 'w') as f:
  f.write('0.927 0.936')